<a href="https://colab.research.google.com/github/nithin-k-mundrathi/python_practice/blob/main/SQL_Nick_Singh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# User's Third Transaction
select user_id,spend,transaction_date from
(SELECT user_id,spend,transaction_date,ROW_NUMBER()
OVER(PARTITION BY user_id order by transaction_date) FROM transactions)
as a
where a.row_number =3

with cte_1 as (
SELECT user_id,spend,transaction_date,
    dense_rank() over(
    partition by user_id order by transaction_date
    ) as rank_dense
FROM transactions
)

select user_id,spend,transaction_date
from cte_1 where  rank_dense =3


;

In [ ]:
# Average Review Ratings
with cte as (

SELECT EXTRACT(MONTH from submit_date) as month_1,*  from reviews
)

select month_1 as month,product_id,CAST(AVG(stars)  AS DECIMAL(5, 2)) as avg_stars
from cte group by month_1, product_id ORDER BY month_1,product_id

In [ ]:
# Final Account Balance

WITH table_1 as (
SELECT account_id,transaction_type,SUM(amount) as amount FROM transactions
group by account_id, transaction_type
)


select a.account_id, a.amount - b.amount  as final_balance from table_1 a
left join table_1 b on  (a.account_id =b.account_id) and (a.transaction_type !=b.transaction_type)
where a.transaction_type = 'Deposit' and b.transaction_type = 'Withdrawal'
order by a.account_id
;

In [ ]:
# Teams Power Users

with month_table as(
select *,EXTRACT(month from sent_date) as date_month,
EXTRACT(year from sent_date) as date_year
FROM messages
)

select sender_id,count(*) as message_count from month_table
where date_month=8 and date_year = 2022
GROUP BY sender_id
order by message_count DESC
LIMIT 2
;

# solution


In [ ]:
# Highest-Grossing Items

with cte as (
select category,product,Sum(spend) as total_spend
from product_spend
where extract(year from transaction_date) = 2022
group by category,product
)

select category,product,total_spend from (select category,product,total_spend,
RANK() OVER (
  PARTITION BY category
  ORDER BY total_spend DESC
)
from cte) T
where rank<3
;

In [ ]:
# Laptop vs Mobile Viewership
with cte as (SELECT count(user_id) as num_users,
  CASE
  when device_type = 'laptop' THEN 'laptop'
  ELSE 'mobile'
 END AS new_device
FROM viewership
group by new_device),

cte_2 as(
select CASE
  when new_device = 'laptop' THEN num_users
 END as laptop_views from cte
),

cte_3 as(
select CASE
  when new_device = 'mobile' THEN num_users
 END as mobile_views from cte
)



select * from cte_2  join cte_3 on 1=1
where laptop_views >0 and mobile_views >0
;


# Solution
SELECT
  COUNT(CASE WHEN device_type = 'laptop' THEN 1 ELSE NULL END) AS laptop_views,
  COUNT(CASE WHEN device_type IN ('tablet', 'phone') THEN 1 ELSE NULL END) AS mobile_views
FROM viewership;

In [ ]:
# Ad Campaign ROAS

select advertiser_id, CAST(revenue_1/spend_1 AS DECIMAL(12,2)) as ROAS from
(
SELECT advertiser_id,SUM(spend) as spend_1,SUM(revenue) as revenue_1 FROM ad_campaigns
group by advertiser_id)
as t_1

order by advertiser_id
;

In [ ]:
# Highest Number of Products


select user_id, product_num from (
select user_id,count(product_id) as product_num,SUM(spend)

from user_transactions
where user_id IN
(SELECT user_id FROM user_transactions
group by user_id
having SUM(spend) >=1000
)

group by user_id
order by count(product_id) Desc,SUM(spend) DESC
fetch first 3 rows only
) as t_1

;

In [ ]:
# Second Day Confirmation

with cte as (
SELECT *,signup_date + INTERVAL '1 day' as new_date FROM texts t
LEFT JOIN emails e on t.email_id = e.email_id
where t.signup_action	= 'Confirmed'
)

select user_id from cte
where new_date = action_date
;


with cteA as (
SELECT * FROM texts where
signup_action = 'Confirmed'
),

cteB as (
select a.email_id,a.user_id,a.signup_date
,b.email_id, b.action_date,
date(b.action_date) - date(a.signup_date) as date_diff
from emails a
left join cteA b
on a.email_id = b.email_id
)


select user_id from cteB
where date_diff =1
;

In [ ]:
# App Click-through Rate (CTR)
with CTEA as (
SELECT *
, extract(year from timestamp)

FROM events
where extract(year from timestamp) = 2022
),


cteB as (
select app_id
, sum(case when event_type= 'click' Then 1
        Else 0
        end) as count_click
, sum(case when event_type= 'impression' Then 1
        Else 0
        end) as count_impression
from CTEA
group by app_id
)

select app_id,
cast(((100.0) * count_click /count_impression) as decimal(5,2)) as CTR
from cteB


;

In [ ]:
# Well Paid Employees

with cte_managers As (
SELECT * FROM employee
where manager_id IS NULL
),

cte_employees AS (
SELECT * FROM employee
where manager_id IS NOT NULL
)

select a.employee_id,a.name AS employee_name from cte_employees a
left join cte_managers b
on a.manager_id = b.employee_id
where a.salary > b.salary
;

In [ ]:
# Top Three Salaries
with cte_a AS (
SELECT department_id,salary,
Dense_rank() Over (
partition by department_id order by salary DESC
)As rank_salary, name
FROM employee
)

select b.department_name,a.name,a.salary from cte_a  a
left join  department b
on a.department_id = b.department_id
where rank_salary <=3
order by 1 Asc, 3 desc , 2 Asc

;

In [ ]:
# Average Post Hiatus (Part 1)
with cte as (
SELECT *
FROM posts
where extract(year from post_date) = 2021
),

 cte_1 as (
select *,
DATE(post_date) as date_1,
Rank() OVER(
  PARTITION BY user_id
  order by post_date
)
from cte
order by user_id
)

select user_id,MAX(date_1)-MIN(date_1) as days_between
from cte_1
group by user_id
having MAX(date_1)-MIN(date_1) > 0
;

In [ ]:
# Top 5 artists

with cte as (
select * from global_song_rank
where rank <=10
order by rank ASC
),

 cte_2 as (
select day,a.song_id,rank,artist_id from cte a left join songs b ON
a.song_id = b.song_id
),

cte_3 as (
select artist_id,count(song_id),
DENSE_RANK() OVER(ORDER BY count(song_id) DESC)from cte_2
group by artist_id
order by count(song_id) Desc
)

select b.artist_name,a.dense_rank as artist_rank from cte_3 a
left join artists b on
a.artist_id = b.artist_id
where a.dense_rank <=5
order by a.dense_rank Asc,b.artist_name;

In [ ]:
# Signup Activation Rate

with cte_a AS (
SELECT CAST(count(1) AS DECIMAL(9,2)) as c_a FROM emails
),

cte_b as (
Select CAST(Count(1) As decimal(9,2)) as c_b from emails a
left join texts b
on a.email_id = b.email_id
where b.signup_action = 'Confirmed'
)


select cast(c_b/c_a As decimal(9,2)) from cte_a a
left join cte_b b
on 1=1
;

In [ ]:
# Supercloud Customer

with cte_a as (
select count(1) FROM (
SELECT distinct(product_category) FROM products ) as t
),

cte_b as (
select customer_id, count(*) as count_1 from
(select distinct customer_id,product_category  from customer_contracts a
  left join products b on a.product_id = b.product_id
) as k
group by customer_id
)

select customer_id from cte_b where count_1 in (select * from cte_a)
-- select distinct customer_id, product_id from customer_contracts
;

In [ ]:
|#Purchasing Activity by Product Type

with cte as (
SELECT order_date, product_type,SUM(quantity) OVER (PARTITION BY product_type order by order_date)
FROM total_trans
)

select order_date,product_type,sum as cum_purchased from cte
order by order_date,cum_purchased
;

In [ ]:
#  3-Topping Pizzas

with cte as (
SELECT CONCAT(t_1.topping_name,',',t_2.topping_name,',',t_3.topping_name) as pizza,
t_1.ingredient_cost + t_2.ingredient_cost + t_3.ingredient_cost as total_cost
FROM pizza_toppings t_1
left join  pizza_toppings t_2
on t_1.topping_name != t_2.topping_name
left join pizza_toppings t_3
on t_1.topping_name != t_3.topping_name and t_2.topping_name != t_3.topping_name
where (t_1.topping_name <= t_2.topping_name) and
(t_2.topping_name <= t_3.topping_name)
ORDER BY t_1.topping_name,t_2.topping_name,t_3.topping_name
)

select  *
from cte
order by total_cost DESC
offset 0 rows
fetch next 3 rows only
;

In [ ]:
# Y-on-Y Growth Rate
with cte as (
SELECT extract(year from transaction_date) as year_1,product_id,
spend as curr_year_spend
FROM user_transactions
),

-- cte_2 as (
-- select product_id,year_1,SUM(spend) as spend from cte
-- group by product_id,year_1
-- )

-- select * from cte_2
-- order by product_id,year_1
cte_2 as (
select *,
LAG(curr_year_spend, 1) OVER (
    PARTITION BY product_id
    ORDER BY year_1) as prev_year_spend
    from cte
)

SELECT *,
CAST(((curr_year_spend-prev_year_spend)/prev_year_spend)*100 as decimal(5,2))
from cte_2
;


with cte_a as  (
SELECT
product_id
,sum(spend) as spend
, extract(year from transaction_date) as year
, dense_rank() over(
  partition by product_id order By
  extract(year from transaction_date) Desc
)
FROM user_transactions
group by extract(year from transaction_date), product_id
),

cte_b AS (
select year,product_id
, spend AS curr_year_spend
,lag(spend,1) Over(
partition by product_id order by dense_rank Desc
) as prev_year_spend
from cte_a
)

select *
, cast(((curr_year_spend - prev_year_spend)*100 / (prev_year_spend)) AS decimal(9,2)) as yoy_rate

from cte_b
;

In [ ]:
# Repeated Payments

with cte as (
SELECT
  transaction_id
  merchant_id,
  credit_card_id,
  amount,
  transaction_timestamp,
  LAG(transaction_timestamp) OVER (
    PARTITION BY merchant_id,
    credit_card_id,
    amount
    ORDER BY
      transaction_timestamp
  ) AS previous_transaction
FROM
  transactions
),

cte_2 as (
select *,
extract(epoch from transaction_timestamp-previous_transaction) /60 as min_time
from cte
where extract(epoch from transaction_timestamp-previous_transaction) /60<=10
)

select count(1) as payment_count from cte_2
;

with cte_a as (
SELECT *
, lag(transaction_timestamp) OVER(
partition by credit_card_id,merchant_id,amount order by transaction_timestamp Asc
) as previous_transaction
FROM transactions
)

SELECT count(1) as payment_count
FROM cte_a
where extract(MINUTE from transaction_timestamp - previous_transaction) <10




In [ ]:
# Server Utilization Time

with cte_1 as (SELECT *,
LAG(status_time) OVER (
    PARTITION BY server_id
    order by status_time ) as stop_time
FROM server_utilization
),

cte_2 as (
select *,
extract(epoch from status_time-stop_time) as time_1
from cte_1
where session_status = 'stop'
)

SELECT cast(SUM(time_1)/86400 as int) as total_uptime_days from cte_2
;


with cte_a AS (
SELECT *,
lag(status_time) over(
partition by server_id order by status_time Asc
) as start_time
FROM server_utilization
),

cte_b as (
select *
,extract(epoch from status_time - start_time)
from cte_a
where session_status = 'stop'
)

select cast(sum(extract)/86400  AS Int) AS total_uptime_days from cte_b

;

In [ ]:
# matching renatl amenities
with cte as (
select rental_id,
        array_agg(amenity order by rental_id) as amenities
from rental_amenities
group by rental_id
)

select count(*) as matching_airbnb
from cte a
left join cte b on a.amenities = b.amenities
where a.rental_id> b.rental_id


In [ ]:
# histogram of tweets

with cte as (
SELECT user_id,count(tweet_id) as user_num FROM tweets
where  Extract(year from  tweet_date) = 2022
GROUP BY user_id
)

select user_num as tweet_bucket,count(user_id) as users_num from cte
group by user_num
;

In [ ]:
# Page with No Likes

with cte as (
SELECT page_id,count(liked_date) as c_1 FROM page_likes
group by page_id
)

select a.page_id from pages a
left join cte b on a.page_id=b.page_id
where c_1 is NULL
;

In [ ]:
# duplicate Job Listings

with cte as (
SELECT count(1) FROM job_listings
group by title,description,company_id
having count(job_id) >1
)

select count(*) as co_w_duplicate_jobs from cte
;

In [ ]:
# Apple Pay Volume

with cte as (
SELECT *,
case when payment_method = 'Apple Pay' then transaction_amount
     when payment_method = 'apple pay' then transaction_amount
     when payment_method = 'apple Pay' then transaction_amount
     when payment_method = 'Apple pay' then transaction_amount
     ELSE 0
     END as apple_transaction
FROM transactions
)

select merchant_id,sum(apple_transaction) as total_transaction from cte
group by merchant_id
order by total_transaction DESC
;

In [ ]:
# Cities With Completed Trades
with cte as (
SELECT * FROM trades
where status = 'Completed'
)

select b.city,count(a.status) as total_orders from cte a left join users b
on a.user_id=b.user_id
group by b.city
order by count(a.status) DESC
offset 0 rows
fetch next 3 rows only
;

In [ ]:
#top Rated Business

with cte as (
SELECT count(distinct business_id) as a_1 FROM reviews
),

cte_2 as (
SELECT count(distinct business_id) as b_1 from reviews
where review_stars =5 or review_stars =4
)

select b_1 AS business_count,
(b_1*100/a_1) as top_rated_pct
from cte_2 a left join  cte b on 1=1
;

In [ ]:
# lInkedIn Power Creators

SELECT pp.profile_id FROM personal_profiles as pp
left join company_pages as cp ON
pp.employer_id = cp.company_id
where pp.followers >= cp.followers
order by pp.profile_id
;


In [ ]:
# Unfinished Parts

with cte as (
SELECT part,count(finish_date) FROM parts_assembly
group by part,assembly_step
having count(finish_date) =0
)

select distinct part from cte
;

In [ ]:
# data science skills
SELECT
  distinct candidate_id
FROM candidates
WHERE
skill IN ('Python', 'Tableau', 'PostgreSQL')
group by candidate_id
having count(skill) >=3
;

In [ ]:
# Spare Server Capacity
with cte as (
SELECT datacenter_id,SUM(monthly_demand) as demand FROM forecasted_demand
group by datacenter_id
)

select a.datacenter_id,
       monthly_capacity - demand as spare_capacity
from datacenters a left join cte b on a.datacenter_id= b.datacenter_id
order by a.datacenter_id
;

In [ ]:
# Repeat Purchases on Multiple Days

with cte as (
SELECT distinct user_id,product_id,DATE(purchase_date) as date_1 FROM purchases
),

ste_2 as (
select distinct user_id from cte
group by user_id,product_id
having count(date_1) >=2
)

SELECT count(*) as repeat_purchasers from ste_2
;

In [ ]:
# First Transaction

with cte as (
SELECT *,
      RANK () OVER (
			PARTITION BY user_id
			ORDER BY transaction_date ASC
		) price_rank
FROM user_transactions
)

select count(distinct user_id) as users from cte
where price_rank= 1 and spend >=50
;

In [ ]:
# Active User Retention

with cte as (
SELECT user_id,event_id,event_type,extract(MONTH from event_date) as month_1
,extract(year from event_date) as year_1 FROM user_actions
where extract(year from event_date) =2022
and (extract(MONTH from event_date) = 7 or extract(MONTH from event_date) =6)
),

cte_2 as (
select a.user_id,a.month_1 from cte a
left join  cte b on  (a.month_1>b.month_1)
where a.user_id=b.user_id
),

cte_3 as (
select count(distinct cte_2.user_id) as monthly_active_users from cte_2
)

select distinct cte_2.month_1 as month,monthly_active_users  from cte_2
left join cte_3 on 1=1
;

with cte_a As  (
SELECT distinct user_id,event_type,
extract(month from event_date) as month,
extract(year from event_date) as year
FROM user_actions
where  extract(month from event_date) IN (7,6) AND
extract(year from event_date) = 2022
),

cte_b as (
select user_id,count(*)
from cte_a
group by user_id
having count(*) >=3
),
cte_c AS (
select distinct month from cte_a where month =7
)

select  month
,count(1) AS monthly_active_users
from cte_b a
left join cte_c b
On 1=1
group by b.month


In [ ]:
# Same Week Purchases

with cte as (
SELECT count(distinct user_id) as c_1 FROM signups
),

cte_2 as (

select count(distinct a.user_id) as c_2 from user_purchases a
left join signups b on a.user_id = b.user_id
where date(a.purchase_date) <= date(b.signup_date + INTERVAL '7 day')
)

select cast(a.c_2*100/b.c_1 as decimal(5,2)) as same_week_purchases_pct from cte_2 a
left join cte b on 1=1
;

In [ ]:
# Signup Confirmation Date

with cte as (
select cast(count(*) as decimal(5,2)) as c_1 from (
select distinct email_id,signup_action from texts
where signup_action	= 'Confirmed'
) t),

cte_2 as (
  select count(distinct(user_id)) as c_2 from emails
)

select cast(c_1/c_2 as decimal(6,2))
as confirmation_rate from cte left join cte_2 on 1=1
;

In [ ]:
# Unique Money Transfer relationships

with cte as(
SELECT count(*) FROM payments a
left join payments b on b.payer_id = a.recipient_id and a.payer_id = b.recipient_id
where b.amount >=0
group by a.payer_id,a.recipient_id
)

select COUNT(1)/2 as unique_relationships from cte
;

In [ ]:
# User Shopping Sprees

with cte as (
SELECT *,LAG(transaction_date, 1) OVER (
    PARTITION BY user_id
    ORDER BY transaction_date) as prev_transaction_date
FROM transactions
),

cte_2 as (
select *,LAG(prev_transaction_date, 1) OVER (
    PARTITION BY user_id
    ORDER BY prev_transaction_date) as prev_2_transaction_date
    from cte
)

select distinct user_id as user_id from cte_2
WHERE prev_2_transaction_date is not NULL and prev_transaction_date is NOT NULL
;

with cte_a AS (
SELECT *,
date(transaction_date) as today_date,
date(transaction_date) + interval '1 day' as second_date,
date(transaction_date) + interval '2 day' as third_date,
lead(date(transaction_date),1) over(
partition by user_id order by date(transaction_date)) as second_day_user,
lead(date(transaction_date),2) over(
partition by user_id order by date(transaction_date)) as third_day_user
FROM transactions
)

select user_id from cte_a
where (second_date = second_day_user) AND (third_date = third_day_user)

;

In [ ]:
# Sending Vs Opening Snaps

with cte as (
SELECT b.age_bucket,SUM(time_spent) as open_perc FROM activities a left join
age_breakdown b on a.user_id=b.user_id
where activity_type='open'
group by b.age_bucket,a.activity_type
),

cte_1 as (
SELECT b.age_bucket,SUM(time_spent) as  send_perc FROM activities a left join
age_breakdown b on a.user_id=b.user_id
where activity_type='send'
group by b.age_bucket,a.activity_type
)


select a.age_bucket,
cast(b.send_perc*100 / (a.open_perc + b.send_perc) as decimal(5,2)) as send_perc ,
cast(a.open_perc*100/(a.open_perc + b.send_perc) as decimal(5,2)) as open_perc
from cte a left join cte_1 b on a.age_bucket=b.age_bucket
;

In [ ]:
# Photoshop revenue Analysis

with cte_1 as (
select * from adobe_transactions
where customer_id in (
SELECT customer_id FROM adobe_transactions
where product = 'Photoshop'))

select customer_id,SUM(revenue) as revenue from cte_1
where product !='Photoshop'
group by customer_id
;

In [ ]:
# # Spotify Streaming History

with cte as (
SELECT user_id,song_id,date(listen_time) FROM songs_weekly
where date(listen_time) <= '08/04/2022'
),

cte_2 as (
select user_id,song_id,count(*) as c_1 from cte a
group by user_id,song_id ),

cte_3 as (
select user_id,song_id,SUM(song_plays) as c_1
from songs_history
group by user_id,song_id),

cte_4 as (
select * from cte_2 union ALL select * from cte_3
)

select user_id,song_id,SUM(c_1) from cte_4
group by user_id,song_id
order by SUM(c_1) DESC
;

In [ ]:
# Invalid Search Results

with cte as (
SELECT *,(num_search*invalid_result_pct)/100 as invalid_searches_pct  FROM search_category
where num_search is NOT NULL and invalid_result_pct is NOT NULL
)

select country, SUM(num_search),
cast(SUM(invalid_searches_pct)*100.0/SUM(num_search) as DECIMAL(5,2)) from cte
group by country
;

In [ ]:
# Consulting Bench Time

with cte as (
SELECT b.employee_id,
        a.start_date,a.end_date,
        a.job_id,a.client_id
,EXTRACT(YEAR from end_date) FROM consulting_engagements a
left join  staffing b on a.job_id = b.job_id
where EXTRACT(YEAR from a.end_date) <=2021 and b.is_consultant='true'
order by  b.employee_id,a.start_date,a.end_date
)

select employee_id,365-SUM(end_date-start_date+1)
    from cte
group by employee_id
;

In [ ]:
# Odd and Even Measurements
with cte as (
SELECT *,date(measurement_time) as date_1,
RANK () OVER (
			PARTITION BY date(measurement_time)
			ORDER BY measurement_time
		) price_rank
FROM measurements
),

cte_2 as (
select *,price_rank%2 as remain,
case when price_rank%2=0 then measurement_value
else 0 END as even_value,
case when price_rank%2=1 then  measurement_value
else 0 END as odd_value
 from cte
)

select date_1, SUM(odd_value) as odd_sum ,
SUM(even_value) as even_sum
from cte_2
group by date_1
;

In [ ]:
# Compensation Outliers

with cte as (
SELECT title,
2*AVG(salary) as two_salary,
AVG(salary)/2 as half_salary
FROM employee_pay
group by title
),

cte_2 as (
select employee_id,salary,
CASE when salary>two_salary then 'Overpaid'
 when salary<half_salary then 'Underpaid'
END as outlier_status
from employee_pay a left join cte b
on a.title=b.title
)

select * from cte_2
where outlier_status is NOT NULL
;

In [ ]:
# Fill Missing Client Data

WITH fill_products
AS (SELECT
  product_id,
  category,
  name,
  COUNT(category) OVER (ORDER BY product_id) AS category_group
FROM products)
SELECT
  product_id,
  FIRST_VALUE (category) OVER (PARTITION BY category_group ORDER by product_id) AS category,
  name
FROM fill_products

In [ ]:
# LinkedIn Power Creators (Part 2)

with cte as (
SELECT a.company_id as a_cid,a.name as a_name,a.followers as a_followers,
b.personal_profile_id as profile_id
FROM company_pages a
left join  employee_company  b
on a.company_id=b.company_id
),

cte_2 as (
select profile_id, SUM(a_followers) as p_followers from cte group by profile_id
)

select distinct a.profile_id from cte_2 a left join personal_profiles b
on a.profile_id=b.profile_id
where a.p_followers < b.followers
order by a.profile_id
;

In [ ]:
# Tweets' Rolling Averages

select user_id,tweet_date,
CAST(avg(tweet_count) over (
partition by user_id order by tweet_date
ROWS between 2 preceding and current row
) AS decimal(9,2)
)AS rolling_avg_3d
from tweets
group by user_id,tweet_date,tweet_count

In [ ]:
# Reactivated Users

with cte as (
SELECT *,EXTRACT(Month from login_date) as month_1,
EXTRACT(Year from login_date) as year_1
FROM user_logins
),
cte_2 as (
select *,LAG(login_date, 1) OVER (
    PARTITION BY user_id
    ORDER BY login_date) as prev_year_spend from cte
),
cte_3 as (
select *,extract(month from prev_year_spend) as month_2,
case when  month_1= extract(month from prev_year_spend)+1 then 0
ELSE 1 END as react
 from cte_2
)

select month_1 as current_month,SUM(react) as reactivated_users from cte_3 group by month_1
having sum(react) >0
order by month_1
;

In [ ]:
# Senior Managers

with cte_emp as (
SELECT DISTINCT emp_id FROM employees
),

cte_m as (
select distinct manager_id from employees
),

cte_4 as (
select manager_id,emp_id from employees
where emp_id in (select * from cte_m)
),

cte_5 as (
select manager_id,count(*) as direct_reportees from employees
where emp_id in (select * from cte_m) and manager_id is NOT NULL
and manager_id in (select emp_id from cte_4)
group by manager_id
)

select distinct b.manager_name,a.direct_reportees
from cte_5 a left join employees b on a.manager_id=b.manager_id
order by a.direct_reportees DESC
;

In [ ]:
#  cards IIssued Difference
with cteA as (
SELECT issue_month,
card_name
,sum(issued_amount) as count_n
FROM monthly_cards_issued
group by issue_month, card_name
)

select distinct card_name
,max(count_n) OVER (partition by card_name) - min(count_n) OVER (partition by card_name) as difference
from cteA
order by 2 Desc
;

In [ ]:
# IBM db2 Product Analytics

with cteA as (
SELECT employee_id, count(query_id) as n_count FROM queries
where
extract(year from query_starttime) = 2023  AND
extract(month from query_starttime) > 6 AND extract(month from query_starttime) < 10
group by employee_id
)

select COALESCE(n_count,0) AS unique_queries
,count(a.employee_id) as employee_count
from employees a
left join cteA  b
on  a.employee_id = b.employee_id
group by n_count
order by COALESCE(n_count,0)
;

In [ ]:
# Compressed Mean
SELECT
cast(
SUM(item_count* order_occurrences)/ SUM(order_occurrences) as decimal(19,1)) as mean

FROM items_per_order;

# Compressed Mode

with cte_a AS (
SELECT order_occurrences,count(*) FROM items_per_order
group by order_occurrences
order by 2 desc
),

cte_b as (
select order_occurrences as mode from cte_a where count in (
select max(count) from cte_a
)
)

select item_count as mode from cte_b a
left join items_per_order b
on a.mode = b.order_occurrences
where order_occurrences in (
select max(order_occurrences) from cte_a
)
order by 1
;

In [ ]:
# International Call Percentage

with cte_a as
(SELECT count(1) as count_a FROM phone_calls
),

cte_b as (
select count(1) AS count_b from (
SELECT a.*, b.country_id,c.country_id FROM phone_calls a
left join phone_info b
on a.caller_id = b.caller_id
left join phone_info c
on a.receiver_id = c.caller_id
where b.country_id != c.country_id
) as t
)

select cast(cast(count_b AS Decimal(9,2))*100/count_a AS Decimal(9,1)) from cte_a
inner join cte_b
on 1=1

;

In [ ]:
# Histogram of Users and Purchases

with cte_a as (
SELECT *
, count(*) over(
partition by user_id,transaction_date order by product_id
) as count_products
, max(transaction_date) over(
partition by user_id order by transaction_date
)

FROM user_transactions
),

cte_b as
(
select max(transaction_date) as transaction_date,
user_id
from cte_a
group by user_id
)


select a.transaction_date, a.user_id, max(count_products) AS purchase_count from cte_a a
inner join cte_b b
on (a.user_id = b.user_id) and (a.transaction_date = b.transaction_date)

group by a.transaction_date, a.user_id
;

In [ ]:
# Patient Support Analysis (Part 2)

with Cte_a AS
(
SELECT count(1) As count_a FROM callers
),

cte_b as  (
select count(1) as count_b from callers
where  (call_category is NULL) OR (call_category = 'n/a')
)

select cast(cast(count_b AS Decimal(9,2))*100/count_a As decimal(9,1)) AS uncategorised_call_pct from cte_a a
left join cte_b b
on 1=1

;

In [ ]:
# Card Launch Success

with cte_a AS (
SELECT *
,dense_rank() over (
partition by card_name order by issue_year,issue_month asc
) as rank_month
FROM monthly_cards_issued
)

select card_name,issued_amount from cte_a
where rank_month =1
order by issued_amount Desc
;

In [ ]:
# Pharmacy Analytics (Part 2)

SELECT
  manufacturer,
  COUNT(drug) AS drug_count,
  ABS(SUM(total_sales - cogs)) AS total_loss
FROM pharmacy_sales
WHERE total_sales - cogs <= 0
GROUP BY manufacturer
ORDER BY total_loss DESC;

In [ ]:
# Pharmacy Analytics (Part 3)


SELECT manufacturer, concat('$' ,concat(cast(SUM(total_sales) /1000000 AS INT), ' million'))
FROM pharmacy_sales
group by manufacturer
order By cast(SUM(total_sales) /1000000 AS INT) Desc, manufacturer Desc
;

In [ ]:
# FAANG

with cte_a AS (
SELECT *,
to_char(to_timestamp(to_char(extract(month from date), '999'), 'MM'), 'Mon') AS month_1 ,
extract(year from date) AS year_1
FROM stock_prices
),

cte_b as (
select DISTINCT ticker,
max(open) over(
partition by ticker) as high_1,
min(open) over(
partition by ticker) as low_1
from cte_a
),

cte_c as (
select b.ticker,b.high_1,concat(a.month_1, '-',a.year_1) AS highest_mth from cte_a a
inner join cte_b b
ON (a.ticker = b.ticker) and ((a.open = b.high_1))
),

cte_d AS (
select b.ticker,b.low_1,concat(a.month_1, '-',a.year_1) as lowest_mth from cte_a a
inner join cte_b b
ON (a.ticker = b.ticker) and ((a.open = b.low_1))
)

select a.ticker,highest_mth,b.high_1 as hightest_open, lowest_mth,a.low_1 as lowest_open
from cte_d a
inner join cte_c b
on a.ticker = b.ticker
order by a.ticker


In [ ]:
# Swapped Food Delivery

with cte_a AS (
SELECT *,
coalesce(lead(item,1) over(
order By order_id),item) as lead_1,
coalesce(lag(item,1) over(
order By order_id),item) as lag_1
FROM orders
)

select order_id AS corrected_order_id,
case when order_id %2 = 0 then lag_1
      else lead_1
      End as item
from cte_a
;

In [ ]:
# SQL Syntax

# extarcty Date from a Column
Extract( day/month/year from date_column)

# CASE Statement
CASE
  WHEN a==b THEN 0
  WHEN b==c THEN 0
  ELSE 1
END AS

# extract date from datetime
DATE(post_date)

# subtract two dates to extarct number of days
max(post_date) - min(post_date)

# ADD One day to date
Date + interval "1 Day"

# Dense rank
dense_rank() over (
    partition by a
    order by b
) as rank_dense

# cast as Decimal
cast(avg(stars) as decimal(5,2)) as avg_stars

# Replace nul WITH 0
coalesce(COLUMN_NAME, 0)

# Dealing with TIES

# Concat string with
concat('$' ,concat(cast(SUM(total_sales) /1000000 AS INT), ' million'))

# Rows between 2 preceding and current row
avg(count(*)) over (
partition by user_id order by tweet_date
ROWS between 2 preceding and current row
) AS rolling_avg_3d